In [1]:
# 0) Instalar todas as dependências necessárias
%pip install langchain langchain-community openai pandas pandasql python-dotenv langchain_openai langchain-core

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Pichau\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# 1) Importação das bibliotecas necessárias
from langchain_openai import ChatOpenAI # Alterado para langchain_openai
from langchain.agents import initialize_agent, AgentType # Usado para os sub-agentes
from langchain_core.tools import StructuredTool # Para ferramentas com schema
from langchain.memory import ConversationBufferMemory # Importação da memória
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder # Para o agente principal
from langchain.agents import AgentExecutor, create_openai_tools_agent # Para o agente principal
from dotenv import load_dotenv
import os
import zipfile
import pandas as pd
from pandasql import sqldf
from io import TextIOWrapper
import re
from pydantic import BaseModel, Field # Necessário para StructuredTool
from langchain_core.messages import HumanMessage, AIMessage 

# Variável global para armazenar o dataframe
df_global = None

In [3]:
# 2) Criação da função das ferramentas
def carregar_arquivos(arquivo_zip_path: str) -> str:
    """
    Quando necessário extrai os arquivos do formato .zip (Extrair,Extract,Descompactar,Unzip,Abrir,Open,Deszipar,Decompress,Desempacotar,Unarchive,Desarquivar)
    Após carregar e mesclar, informa o número de linhas, colunas e a lista das colunas disponíveis.
    Exemplo: 'C:\\Users\Pichau\Downloads\\teste\\202401_NFs.zip'
    """
    arquivo_zip_path = arquivo_zip_path.strip("'\"")
    global df_global # Declara que você vai modificar a variável global
    grupo_csv = {}
    
    try:
        with zipfile.ZipFile(arquivo_zip_path, 'r') as zip_ref:
            csv_files_in_zip = [f for f in zip_ref.namelist() if f.endswith('.csv')]
            
            if len(csv_files_in_zip) < 2:
                return f"Erro: O arquivo ZIP '{os.path.basename(arquivo_zip_path)}' precisa conter pelo menos dois arquivos CSV para serem mesclados. Foram encontrados {len(csv_files_in_zip)} CSVs."
            
            for arquivo_csv in csv_files_in_zip:
                with zip_ref.open(arquivo_csv) as arq:
                    df = pd.read_csv(TextIOWrapper(arq,'utf-8', errors='ignore'))
                    grupo_csv[arquivo_csv] = df
            
        sorted_keys = sorted(grupo_csv.keys())
        primeiro_df = grupo_csv[sorted_keys[0]]
        segundo_df = grupo_csv[sorted_keys[1]]
        
        # Tentativa de merge mais robusta: verificar se 'CHAVE DE ACESSO' existe em ambos
        if 'CHAVE DE ACESSO' not in primeiro_df.columns or 'CHAVE DE ACESSO' not in segundo_df.columns:
            return f"Erro: A coluna 'CHAVE DE ACESSO' não foi encontrada em um ou ambos os arquivos CSV ({sorted_keys[0]}, {sorted_keys[1]}). Não foi possível mesclar."

        df_global = pd.merge(primeiro_df, segundo_df, on='CHAVE DE ACESSO', how='left')
        df_global.columns = df_global.columns.str.replace('_x$', '', regex=True)
        
        # Remover colunas _y de forma mais segura
        cols_to_drop_y = [col for col in df_global.columns if col.endswith('_y')]
        # Lista de colunas fixas a serem removidas (se existirem e terminarem com _y)
        fixed_y_cols = ['MODELO_y','SÉRIE_y','NÚMERO_y','NATUREZA DA OPERAÇÃO_y','DATA EMISSÃO_y','CPF/CNPJ Emitente_y','RAZÃO SOCIAL EMITENTE_y','INSCRIÇÃO ESTADUAL EMITENTE_y','UF EMITENTE_y','MUNICÍPIO EMITENTE_y','CNPJ DESTINATÁRIO_y','NOME DESTINATÁRIO_y','UF DESTINATÁRIO_y','INDICADOR IE DESTINATÁRIO_y','DESTINO DA OPERAÇÃO_y','CONSUMIDOR FINAL_y','PRESENÇA DO COMPRADOR_y']
        
        # Filtra as colunas que realmente existem no DataFrame antes de dropar
        cols_to_drop_final = [col for col in (cols_to_drop_y + fixed_y_cols) if col in df_global.columns]
        df_global = df_global.drop(columns=list(set(cols_to_drop_final))) # Usar set para evitar duplicatas e converter para lista
        
        return f"Arquivos carregados e mesclados com sucesso! DataFrame pronto para análise com {len(df_global)} linhas e {len(df_global.columns)} colunas. As colunas disponíveis são: {list(df_global.columns)}."
    except FileNotFoundError:
        return f"Erro: O arquivo ZIP '{arquivo_zip_path}' não foi encontrado. Verifique o caminho."
    except zipfile.BadZipFile:
        return f"Erro: O arquivo '{arquivo_zip_path}' não é um arquivo ZIP válido ou está corrompido."
    except Exception as e:
        return f"Erro inesperado ao carregar ou processar o arquivo ZIP: {e}. Detalhes: {str(e)}"

def consulta_sql(query: str) -> str:
    """
    Executa uma consulta SQL no DataFrame df_global.
    Exemplo: SELECT SUM("VALOR TOTAL") FROM df_global WHERE "CPF/CNPJ DESTINATÁRIO" = '12.345.678/0001-90'
    """
    global df_global
    if df_global is None:
        return "O DataFrame global ainda não foi carregado. Por favor, use a ferramenta 'carregar_arquivos' primeiro."

    try:
        # Prepara o ambiente para pandasql, garantindo que df_global esteja acessível
        globals_dict = {'df_global': df_global}

        result = sqldf(query, globals_dict)
        if result.empty:
            return "A consulta SQL não retornou resultados."
        return result.to_string(index=False)
    except Exception as e:
        # Mensagem de erro mais detalhada
        cols_info = ""
        if df_global is not None:
            cols_info = f"\nColunas disponíveis: {list(df_global.columns)}."
            # Tentar inferir se o erro é de tipo de dado ou coluna inexistente
            if "no such column" in str(e).lower():
                cols_info += " Verifique se os nomes das colunas na sua consulta estão corretos e correspondem exatamente aos nomes listados."
            elif "no such table" in str(e).lower() and 'df_global' not in query.lower():
                cols_info += " Lembre-se de referenciar o DataFrame como 'df_global' na sua consulta SQL."
            elif "could not convert string to float" in str(e).lower() or "numeric" in str(e).lower():
                 cols_info += " Parece que você está tentando realizar uma operação numérica em uma coluna não numérica. Considere usar CAST() no SQL ou verificar o tipo da coluna com 'inspecionar_dataframe'."

        return f"Erro na consulta SQL: {e}.{cols_info}"
    
def mapear_colunas(args: str = None) -> str: # O argumento 'args' é genérico para o agente poder passar qualquer coisa
    """
    Mapeia os nomes das colunas do DataFrame global (df_global) para nomes mais simples e padronizados.
    Esta ferramenta deve ser usada quando houver dificuldade em encontrar nomes de colunas ou consultas SQL falharem por nomes incorretos.
    Não precisa de argumentos específicos, apenas chame a ferramenta para que ela renomeie as colunas.
    """
    global df_global
    if df_global is None:
        return "O DataFrame global (df_global) ainda não foi carregado. Por favor, use a ferramenta 'carregar_arquivos' primeiro."

    colunas_originais = df_global.columns.tolist()
    
    mapeamento = {}
    for col in colunas_originais:
        col_nova = col.upper()
        col_nova = re.sub(r"[^A-Z0-9]", "_", col_nova)  # Substitui espaços/símbolos por _
        col_nova = re.sub(r"_+", "_", col_nova).strip("_")  # Remove múltiplos _
        mapeamento[col] = col_nova

    df_global.columns = [mapeamento[col] for col in df_global.columns]
    
    retorno = "\n".join([f"{orig} -> {novo}" for orig, novo in mapeamento.items()])
    return f"Colunas renomeadas com sucesso:\n{retorno}\nAgora você pode tentar sua consulta SQL novamente com os novos nomes de coluna."

def inspecionar_dataframe(args: str = None) -> str:
    """
    Retorna informações sobre o DataFrame global, incluindo as primeiras linhas,
    nomes das colunas e seus respectivos tipos de dados (dtypes).
    Útil para depurar consultas SQL, entender a estrutura dos dados ou verificar se o DataFrame foi carregado.
    Não precisa de argumentos específicos, apenas chame a ferramenta para inspecionar.
    """
    global df_global
    if df_global is None:
        return "O DataFrame global ainda não foi carregado. Por favor, use a ferramenta 'carregar_arquivos' primeiro."
    
    info = f"DataFrame carregado com {len(df_global)} linhas e {len(df_global.columns)} colunas.\n"
    info += "Colunas e Tipos de Dados:\n"
    for col, dtype in df_global.dtypes.items():
        info += f"- '{col}': {dtype}\n"
    info += "\nPrimeiras 5 linhas:\n"
    # Adicionando um fallback caso o dataframe esteja vazio
    if not df_global.empty:
        info += df_global.head().to_string()
    else:
        info += "O DataFrame está vazio."
    return info

load_dotenv() # carrega o segredo

True

In [4]:
# 3) Escolhe o modelo selecionado e configuração de temperatura
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.25,
    api_key = os.getenv("api_key"),
    max_tokens=80
)

# --- Definição dos Schemas de Entrada para StructuredTool --- Essas classes, construídas com o Pydantic (BaseModel, Field), 
# são cruciais para a comunicação entre o LLM e suas ferramentas. 
# elas atuam como um "contrato" para cada ferramenta
class CarregarArquivosInput(BaseModel):
    arquivo_zip_path: str = Field(description="O caminho completo para o arquivo .zip a ser carregado.")

class ConsultaSQLInput(BaseModel):
    query: str = Field(description="A consulta SQL a ser executada no DataFrame global (df_global).")

class MapearColunasInput(BaseModel):
    args: str = Field(description="Argumento genérico, não é usado diretamente pela função, apenas para conformidade com StructuredTool.")

class InspecionarDataFrameInput(BaseModel):
    args: str = Field(description="Argumento genérico, não é usado diretamente pela função, apenas para conformidade com StructuredTool.")


### Definição dos Sub-Agentes

# Ferramentas específicas para cada sub-agente usando StructuredTool
zip_tool = StructuredTool.from_function(
    func=carregar_arquivos,
    name="carregar_arquivos",
    description=carregar_arquivos.__doc__.strip(),
    args_schema=CarregarArquivosInput
)

query_tool = StructuredTool.from_function(
    func=consulta_sql,
    name="consulta_sql",
    description=consulta_sql.__doc__.strip(),
    args_schema=ConsultaSQLInput
)

map_tool = StructuredTool.from_function(
    func=mapear_colunas,
    name='mapear_colunas',
    description=mapear_colunas.__doc__.strip(),
    args_schema=MapearColunasInput
)

inspect_tool = StructuredTool.from_function(
    func=inspecionar_dataframe,
    name='inspecionar_dataframe',
    description=inspecionar_dataframe.__doc__.strip(),
    args_schema=InspecionarDataFrameInput
)

In [5]:
# 4) Criando os Sub-Agentes com memória
# A memória é adicionada para cada sub-agente e para o agente principal
main_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


# Sub-Agente para Carregamento de Arquivos
agent_carregar = initialize_agent(
    tools=[zip_tool],
    llm=llm,
    agent_type=AgentType.OPENAI_FUNCTIONS, # Usar OPENAI_FUNCTIONS para StructuredTool
    verbose=True,
    handle_parsing_errors=True,
    memory=main_memory, # Adicionando memória
    agent_kwargs={
        "system_message": "Você é um agente especialista em carregar e extrair arquivos ZIP que contêm CSVs. Use a ferramenta 'carregar_arquivos'. Forneça o caminho completo para o arquivo ZIP."
    }
)

# Sub-Agente para Consulta SQL (com a nova ferramenta e prompt aprimorado)
agent_sql = initialize_agent(
    tools=[query_tool, inspect_tool], # Adicione a ferramenta de inspeção aqui
    llm=llm,
    agent_type=AgentType.OPENAI_FUNCTIONS, # Usar OPENAI_FUNCTIONS para StructuredTool
    verbose=True,
    handle_parsing_errors=True,
    memory=main_memory, # Adicionando memória
    agent_kwargs={
        "system_message": """Você é um agente especialista em executar consultas SQL no DataFrame global.
        Use a ferramenta 'consulta_sql'. Formule as consultas SQL com o nome da tabela como 'df_global'.
        
        **Instruções Importantes:**
        0. **Sobre seleção de colunas:** Garanta que na seleção de colunas "SELECT" nenhum tipo de aspas seja utilizado (nem simples, nem duplas)
        1.  **Operações Numéricas:** Ao lidar com operações como 'maior valor', 'menor valor', 'soma', 'média', etc., garanta que a coluna referenciada na consulta SQL seja tratada como um valor numérico. Se a coluna for originalmente texto ou string, use `CAST(coluna AS REAL)` ou `CAST(coluna AS INTEGER)` no SQL para conversão explícita, se necessário.
        2.  **Verificação de Dados:** Se você tiver dúvidas sobre os nomes das colunas, seus tipos de dados, ou para verificar se o DataFrame está carregado, **utilize a ferramenta 'inspecionar_dataframe' antes de formular a consulta SQL**.
        3.  **Tratamento de Erros:** Se uma consulta SQL falhar, analise a mensagem de erro cuidadosamente. Se indicar um problema com nomes de colunas ou tipos de dados, use 'inspecionar_dataframe' ou sugira ao agente principal que tente 'mapear_colunas'.
        """
    }
)

# Sub-Agente para Mapeamento de Colunas
agent_mapear = initialize_agent(
    tools=[map_tool],
    llm=llm,
    agent_type=AgentType.OPENAI_FUNCTIONS, # Usar OPENAI_FUNCTIONS para StructuredTool
    verbose=True,
    handle_parsing_errors=True,
    memory=main_memory, # Adicionando memória
    agent_kwargs={
        "system_message": "Você é um agente especialista em mapear e padronizar nomes de colunas no DataFrame. Use a ferramenta 'mapear_colunas' quando solicitado para simplificar nomes de colunas ou quando o agente principal indicar um problema de nome de coluna."
    }
)



C:\Users\Pichau\AppData\Local\Temp\ipykernel_1840\1304367484.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  main_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
C:\Users\Pichau\AppData\Local\Temp\ipykernel_1840\1304367484.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_car

In [6]:
### 5) Definição das Ferramentas para o Agente Principal (que são os Sub-Agentes)

# Use funções wrapper para adaptar o input para os sub-agentes
def wrapper_carregar_arquivos_agent(arquivo_zip_path: str) -> str:
    # O sub-agente agent_carregar.run espera um dicionário com a chave 'input'
    return agent_carregar.run(input=arquivo_zip_path) # Ou {'input': arquivo_zip_path} dependendo da versão exata

def wrapper_consulta_sql_agent(query: str) -> str:
    # O sub-agente agent_sql.run espera um dicionário com a chave 'input'
    return agent_sql.run(input=query)

def wrapper_mapear_colunas_agent(args: str = None) -> str:
    # O sub-agente agent_mapear.run espera um dicionário com a chave 'input'
    # Como mapear_colunas não usa args, podemos passar um placeholder ou vazio
    return agent_mapear.run(input="mapear colunas") # Ou um valor dummy, se o sub-agente esperar algo.

# Ferramentas do agente principal agora apontam para as funções wrapper
tool_carregar_arquivos_agent = StructuredTool.from_function(
    func=wrapper_carregar_arquivos_agent, # Use a função wrapper aqui
    name="agente_carregar_arquivos",
    description="Útil para extrair, descompactar, abrir ou carregar arquivos de dados no formato .zip. Sempre forneça o caminho completo do arquivo ZIP. Esta deve ser a primeira ferramenta usada se o DataFrame não estiver carregado.",
    args_schema=CarregarArquivosInput
)

tool_consulta_sql_agent = StructuredTool.from_function(
    func=wrapper_consulta_sql_agent, # Use a função wrapper aqui
    name="agente_consulta_sql",
    description="Útil para realizar consultas, análises ou extrair informações do DataFrame já carregado usando SQL (ex: 'qual o valor', 'liste', 'some', 'agrupe por', 'maior', 'menor', 'média'). Esta ferramenta é usada após o carregamento do arquivo. Se houver dúvidas sobre colunas ou tipos, ela pode inspecionar o DataFrame.",
    args_schema=ConsultaSQLInput
)

tool_mapear_colunas_agent = StructuredTool.from_function(
    func=wrapper_mapear_colunas_agent, # Use a função wrapper aqui
    name="agente_mapear_colunas",
    description="Útil quando as consultas falham devido a nomes de colunas complexos ou quando o usuário pede para renomear, padronizar ou listar nomes de colunas. Não precisa de argumentos, apenas chame para mapear.",
    args_schema=MapearColunasInput
)


In [7]:
# 6) Inicializando o Agente Principal (Usando create_openai_tools_agent)
# A memória é passada através do MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """Você é um assistente de dados muito experiente e inteligente.
        Sua principal tarefa é entender a intenção do usuário e delegar a tarefa para o agente especialista mais adequado.
        
        **Fluxo de Trabalho Prioritário:**
        1.  **Carregamento:** Se o DataFrame global (df_global) não foi carregado OU o usuário explicitamente pede para abrir/extrair um arquivo ZIP, use 'agente_carregar_arquivos'.
        2.  **Mapeamento de Colunas:** Faça o mapeamento das colunas imediatamente para auxiliar os agentes no entendimento das colunas e facilitar as queries.
        3.  **Consulta/Análise:** Uma vez que o DataFrame esteja carregado e o usuário faça perguntas que exijam análise de dados (ex: "qual o valor", "liste", "some", "agrupe por", "maior", "menor", "média"), use 'agente_consulta_sql'.
            * **IMPORTANTE:** Se a 'agente_consulta_sql' falhar ou retornar resultados inesperados para operações numéricas, ou se houver dúvidas sobre os dados, o 'agente_consulta_sql' pode usar a ferramenta interna 'inspecionar_dataframe'. Você, como agente principal, não chama 'inspecionar_dataframe' diretamente, mas pode instruir 'agente_consulta_sql' a fazê-lo se necessário.
        
        
        Sempre mantenha o contexto da conversa. Se um arquivo já foi carregado, não peça para carregá-lo novamente para perguntas subsequentes sobre os mesmos dados.
        """),
        MessagesPlaceholder(variable_name="chat_history"), # Para passar o histórico de conversas
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"), # Para o raciocínio do agente
    ]
)

# Ferramentas para o agente principal (os sub-agentes encapsulados)
main_agent_tools = [tool_carregar_arquivos_agent, tool_consulta_sql_agent, tool_mapear_colunas_agent] # Adicionar tool_inspecionar_dataframe_agent aqui se for um sub-agente separado

agent_executor = AgentExecutor(
    agent=create_openai_tools_agent(llm, main_agent_tools, prompt), # Crie o agente usando a nova forma
    tools=main_agent_tools,
    verbose=True,
    handle_parsing_errors=True
)



In [8]:
# 7) Testes
# Inicializa o histórico de chat
chat_history = []

print("--- Cenário 1: Carregar arquivo e fazer primeira consulta ---")
# Usuário carrega o arquivo e faz uma pergunta. O agente deve carregar e depois consultar.
user_input_1 = "Extraia o arquivo localizado em: 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip'. Qual o menor valor da coluna 'VALOR TOTAL'?"
print(f"\nUsuário: {user_input_1}")
response_1 = agent_executor.invoke({"input": user_input_1, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=user_input_1), AIMessage(content=response_1["output"])])
print(f"Resposta: {response_1['output']}\n")


--- Cenário 1: Carregar arquivo e fazer primeira consulta ---

Usuário: Extraia o arquivo localizado em: 'C:\Users\Pichau\Downloads\teste\202401_NFs.zip'. Qual o menor valor da coluna 'VALOR TOTAL'?


> Entering new AgentExecutor chain...

Invoking: `agente_carregar_arquivos` with `{'arquivo_zip_path': 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip'}`




> Entering new AgentExecutor chain...


C:\Users\Pichau\AppData\Local\Temp\ipykernel_1840\3706581695.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return agent_carregar.run(input=arquivo_zip_path) # Ou {'input': arquivo_zip_path} dependendo da versão exata


To answer the question, I need to extract and load the contents of the specified .zip file to understand its structure and contents. This will allow me to provide information about the number of rows, columns, and the list of available columns.

Action: carregar_arquivos
Action Input: C:\Users\Pichau\Downloads\teste\202401_NFs.zip
Observation: Arquivos carregados e mesclados com sucesso! DataFrame pronto para análise com 565 linhas e 30 colunas. As colunas disponíveis são: ['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO', 'DATA EMISSÃO', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE', 'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE', 'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO', 'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL', 'PRESENÇA DO COMPRADOR', 'VALOR NOTA FISCAL', 'NÚMERO PRODUTO', 'DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'NCM/SH (TIPO DE 

In [9]:

print("--- Cenário 2: Pergunta subsequente sem recarregar o arquivo ---")
# O agente deve lembrar que o arquivo já está carregado e ir direto para a consulta.
user_input_2 = "Qual o maior valor da coluna 'VALOR TOTAL'?"
print(f"\nUsuário: {user_input_2}")
response_2 = agent_executor.invoke({"input": user_input_2, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=user_input_2), AIMessage(content=response_2["output"])])
print(f"Resposta: {response_2['output']}\n")



--- Cenário 2: Pergunta subsequente sem recarregar o arquivo ---

Usuário: Qual o maior valor da coluna 'VALOR TOTAL'?


> Entering new AgentExecutor chain...

Invoking: `agente_consulta_sql` with `{'query': 'SELECT MAX(`VALOR TOTAL`) AS maior_valor FROM df_global'}`




> Entering new AgentExecutor chain...
To answer this question, I need to execute an SQL query to find the maximum value in the "VALOR TOTAL" column of the DataFrame `df_global`. First, I should inspect the DataFrame to ensure it is loaded and to understand its structure.

Action: inspecionar_dataframe
Action Input: None
Observation: DataFrame carregado com 565 linhas e 30 colunas.
Colunas e Tipos de Dados:
- 'CHAVE_DE_ACESSO': object
- 'MODELO': object
- 'S_RIE': int64
- 'N_MERO': int64
- 'NATUREZA_DA_OPERA_O': object
- 'DATA_EMISS_O': object
- 'EVENTO_MAIS_RECENTE': object
- 'DATA_HORA_EVENTO_MAIS_RECENTE': object
- 'CPF_CNPJ_EMITENTE': object
- 'RAZ_O_SOCIAL_EMITENTE': object
- 'INSCRI_O_ESTADUAL_EMITENTE': int64
- '

In [10]:
print("--- Cenário 3: Consulta complexa com potencial de tipo de dado ---")
# Teste para o problema de "Nota Fiscal" sendo textual.
# O LLM agora é instruído a tentar CAST se suspeitar de tipo de dado.
user_input_3 = "Qual a 'RAZÃO SOCIAL EMITENTE' que teve o segundo maior 'VALOR TOTAL'?"
print(f"\nUsuário: {user_input_3}")
response_3 = agent_executor.invoke({"input": user_input_3, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=user_input_3), AIMessage(content=response_3["output"])])
print(f"Resposta: {response_3['output']}\n")



--- Cenário 3: Consulta complexa com potencial de tipo de dado ---

Usuário: Qual a 'RAZÃO SOCIAL EMITENTE' que teve o segundo maior 'VALOR TOTAL'?


> Entering new AgentExecutor chain...

Invoking: `agente_consulta_sql` with `{'query': 'SELECT "RAZÃO SOCIAL EMITENTE", "VALOR TOTAL" FROM df_global ORDER BY "VALOR TOTAL" DESC LIMIT 2'}`




> Entering new AgentExecutor chain...
To answer this question, I need to execute the provided SQL query to retrieve the top two entries from the DataFrame `df_global` based on the "VALOR TOTAL" column in descending order. 

Action: consulta_sql
Action Input: SELECT "RAZÃO SOCIAL EMITENTE", "VALOR TOTAL" FROM df_global ORDER BY "VALOR TOTAL" DESC LIMIT 2
Observation
Observation: Erro na consulta SQL: (sqlite3.OperationalError) near "Observation": syntax error
[SQL: SELECT "RAZÃO SOCIAL EMITENTE", "VALOR TOTAL" FROM df_global ORDER BY "VALOR TOTAL" DESC LIMIT 2
Observation]
(Background on this error at: https://sqlalche.me/e/20/e3q8).
Colunas disponíve

In [13]:
print("--- Cenário 4: Solicitação de mapeamento de colunas com recuperação de erro ---")
# Força o uso do mapeamento se a primeira tentativa falhar.
user_input_4 = "Qual 'Tipo de produto' teve maior 'quantidade'?"
print(f"\nUsuário: {user_input_4}")
response_4 = agent_executor.invoke({"input": user_input_4, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=user_input_4), AIMessage(content=response_4["output"])])
print(f"Resposta: {response_4['output']}\n")




--- Cenário 4: Solicitação de mapeamento de colunas com recuperação de erro ---

Usuário: Qual 'Tipo de produto' teve maior 'quantidade'?


> Entering new AgentExecutor chain...

Invoking: `agente_consulta_sql` with `{'query': 'SELECT "Tipo de produto", SUM(quantidade) as total_quantidade FROM df_global GROUP BY "Tipo de produto" ORDER BY total_quantidade DESC LIMIT 1;'}`




> Entering new AgentExecutor chain...
To answer this question, I need to execute the SQL query provided to find out which product type has the highest total quantity. First, I need to inspect the DataFrame to ensure it is loaded and to understand its structure.

Action: inspecionar_dataframe
Action Input: None
Observation: DataFrame carregado com 565 linhas e 30 colunas.
Colunas e Tipos de Dados:
- 'CHAVE_DE_ACESSO': object
- 'MODELO': object
- 'S_RIE': int64
- 'N_MERO': int64
- 'NATUREZA_DA_OPERA_O': object
- 'DATA_EMISS_O': object
- 'EVENTO_MAIS_RECENTE': object
- 'DATA_HORA_EVENTO_MAIS_RECENTE': object
- 'CPF_C